# This is an example with only ONE XML file
## multiblastXML.ipynb is the final version of this code

In [2]:
from Bio.Blast import NCBIXML

In [3]:
blast = NCBIXML.parse(open('/Users/brisbin/desktop/FPKMfilter.xml','rU'))

geneIDlist=[]
giNUMlist=[]
for record in blast:
    if record.alignments:
        #print gene id
         geneIDlist.append(record.query)
        # to print the "best" matches title
         giNUMlist.append(record.alignments[0].title)
        # to print the query sequence
        #print record.alignments[0].hsps[0].query
        
        #'hsps.query' is not the raw sequence. The 'record.query' will give the gene id which can be 
        # used to attain the raw sequence
        #use seqIO to get raw seuqence from the gene_id

len(geneIDlist)

42

In [42]:
geneIDlist[1:3]

[u'TRINITY_DN80050_c0_g2_i1 len=2343 path=[4811:0-408 4828:409-629 4818:630-653 4827:654-1080 4829:1081-1104 4830:1105-1105 4816:1106-2342] [-1, 4811, 4828, 4818, 4827, 4829, 4830, 4816, -2]',
 u'TRINITY_DN97813_c0_g1_i2 len=1946 path=[4282:0-1018 4312:1019-1249 4296:1250-1273 4313:1274-1380 4311:1381-1490 4315:1491-1491 4307:1492-1588 4285:1589-1612 4286:1613-1945] [-1, 4282, 4312, 4296, 4313, 4311, 4315, 4307, 4285, 4286, -2]']

In [4]:
# list of hit titles - includes the gi number, ACN, and species
giNUMlist[1:3]

[u'gnl|BL_ORD_ID|50254456 gi|734606819|ref|XP_010730570.1| PREDICTED: cytoplasmic dynein 2 heavy chain 1 isoform X1 [Larimichthys crocea]',
 u'gnl|BL_ORD_ID|41475618 gi|657556813|ref|XP_008282673.1| PREDICTED: uncharacterized protein LOC103359216 [Stegastes partitus]']

In [44]:
len(giNUMlist)

42

In [46]:
# strip  hit title to give a list of just the gi numbers
newGIlist=[]
for i in range(0,len(giNUMlist)):
    newGI= (giNUMlist[i].split("|")[3])
    newGIlist.append(newGI)
  
newGIlist[0:5]

[u'551507710', u'734606819', u'657556813', u'657541701', u'657556096']

In [47]:
# strip query title to include ony the gene_id
newGENElist=[]
for i in range(0,len(geneIDlist)):
    newGENE= (geneIDlist[i].split(" ")[0])
    newGENElist.append(newGENE)
  
newGENElist[0:5]

[u'TRINITY_DN98892_c3_g2_i2',
 u'TRINITY_DN80050_c0_g2_i1',
 u'TRINITY_DN97813_c0_g1_i2',
 u'TRINITY_DN96404_c0_g1_i1',
 u'TRINITY_DN55485_c0_g1_i1']

In [10]:
## strip the hit title list to include a list of accession numbers only
ACNlist=[]
for i in range(0,len(giNUMlist)):
    newACN= (giNUMlist[i].split("|")[5])
    ACNlist.append(newACN)
  
ACNlist[0:3]

[u'XP_005805384.1', u'XP_010730570.1', u'XP_008282673.1']

If using entrez to search taxonomy database: gi or ACN will not return results. Need to search with species. 

In [48]:
#instead of gi or ACN, need species from hit title. 
#Create new list with the species from each hit. 

## Species
SPlist=[]
for i in range(0,len(giNUMlist)):
    newSP= (giNUMlist[i].split("|")[6])
    SPlist.append(newSP)

## Species Continued
SPlist2=[]
for i in range(0,len(giNUMlist)):
    newSP= (SPlist[i].split("[")[-1])
    SPlist2.append(newSP)

#Species list finished
 
SPlist3=[]
for i in range(0,len(giNUMlist)):
    newSP= (SPlist2[i].split("]")[0])
    SPlist3.append(newSP)
    
SPlist3

[u'Xiphophorus maculatus',
 u'Larimichthys crocea',
 u'Stegastes partitus',
 u'Stegastes partitus',
 u'Stegastes partitus',
 u'Photobacterium leiognathi',
 u'Larimichthys crocea',
 u'Oreochromis niloticus',
 u'Larimichthys crocea',
 u'Larimichthys crocea',
 u'Neolamprologus brichardi',
 u'Poecilia formosa',
 u'Larimichthys crocea',
 u'Scleropages formosus',
 u'Haplochromis burtoni',
 u'Oreochromis niloticus',
 u'Plasmodium reichenowi',
 u'Neolamprologus brichardi',
 u'Oryzias latipes',
 u'Haplochromis burtoni',
 u'Photobacterium leiognathi',
 u'Takifugu rubripes',
 u'Salinivibrio socompensis',
 u'A Chain A, Solution Structure Of The Rpr Domain Of Putative Rna- Binding Protein 16',
 u'Photobacterium leiognathi',
 u'Larimichthys crocea',
 u'Astyanax mexicanus',
 u'Larimichthys crocea',
 u'Larimichthys crocea',
 u'Photobacterium leiognathi lrivu.4.1',
 u'Notothenia coriiceps',
 u'Larimichthys crocea',
 u'Fundulus heteroclitus',
 u'Photobacterium leiognathi',
 u'Stegastes partitus',
 u'Lar

This worked for all entries except one, which did not provide a species in the hit title

In [53]:
# create a dictionary with the key as the gene id and the value as the species name
GeneSpeciesDict={}

for i in range(0,len(newGENElist)):
    newkey = newGENElist[i]
    GeneSpeciesDict[newkey]=SPlist3[i]

len(GeneSpeciesDict)

42

In [62]:
#Get rid of geneIDs that do not have taxonomy available (no species name in title)
# for now - doing this by removing entries whose title is too long (>50 characters)
#to be a species name

#function to look at slice of a dictionary
import itertools
def glance(d):
    return dict(itertools.islice(d.iteritems(), 3))

#keylist is list of 42 trinID that are keys from GeneSpeciesDict - remember this specis list 
# IS repetitive
# if any of the species names are too long - delete entry
keylist = GeneSpeciesDict.keys()
for i in keylist:
    if i in GeneSpeciesDict:
        if len(GeneSpeciesDict[i])> 50:
            del GeneSpeciesDict[i] 
            
len(GeneSpeciesDict)
#deleted the entry "predicted protein"

41

In [63]:
glance(GeneSpeciesDict)

{u'TRINITY_DN106845_c0_g1_i1': u'Larimichthys crocea',
 u'TRINITY_DN15043_c0_g1_i1': u'Photobacterium leiognathi',
 u'TRINITY_DN97813_c0_g1_i2': u'Stegastes partitus'}

In [54]:
# use SPlist3 to fetch TAXID from entrez
#get a NONREPETITIVE dictionary with species name: tax id number

from Bio import Entrez
Entrez.email="margaret.marsbrisbin@oist.jp"

taxID = {}

for i in SPlist3:
    if i in taxID:
        continue
    handle = Entrez.esearch(db="taxonomy", term= i)
    record = Entrez.read(handle)
    #if not record["IdList"]:
    #    pdb.set_trace()
    if record["IdList"]:
        taxID[i] = record["IdList"][0]
#    taxIDlist.append(record["IdList"])
    
    

In [57]:
len(taxID)

20

In [64]:
glance(taxID)

{u'Haplochromis burtoni': '8153',
 u'Neolamprologus brichardi': '32507',
 u'Salinivibrio socompensis': '1510206'}

In [13]:
# experimenting with efetch to get linneage information
# provide tax ID and return linneage information
#### DID NOT USE THIS FOR FINAL LISTS

from Bio import Entrez
Entrez.email="margaret.marsbrisbin@oist.jp"

def get_tax_data(taxid):
    """once we have the taxid, we can fetch the record"""
    search = Entrez.efetch(id = taxid, db = "taxonomy", retmode = "xml")
    return Entrez.read(search)

data=get_tax_data('144197')
#lineage = {d['Rank']:d['ScientificName'] for d in 
    #data[0]['LineageEx'] if d['Rank'] in ['family', 'order']}

#data is a lot of text
# this will be slower than ete because it queries NCBI each time

In [50]:
# an alternative method to get linneage information
#ete2 module for python

#first creates a local taxonomy database. When running in the future, it will check for this 
#database rather than creating it

# FASTER than Entrez efetch

from ete2 import NCBITaxa
ncbi = NCBITaxa()

In [56]:
##use ete to get linneage information for each taxID in dictionary ' taxID'

lineageDict={}

for i in taxID:
    lineage = ncbi.get_lineage(taxID[i])
    names = ncbi.get_taxid_translator(lineage)
    key = taxID[i]
    lineageDict[key]= [names[taxid] for taxid in lineage]

len(lineageDict)

20

# USE dictionaries .....not lists

In [16]:
# make a dictionary for Fish or Not Fish by tax id
# 0 = Not Fish, 1 = Fish
fishDict = {} 

keylist=lineageDict.keys()

for i in keylist:
    key = i
    value = lineageDict[key]
    if 'Teleostei' in value:
        binary = 1
    else:
        binary = 0
    fishDict[key]= binary
  
fishDict


{'113540': 1,
 '1248232': 0,
 '144197': 1,
 '1510206': 0,
 '215358': 1,
 '31033': 1,
 '32507': 1,
 '48698': 1,
 '553611': 0,
 '5854': 0,
 '680': 0,
 '7994': 1,
 '8030': 1,
 '8049': 1,
 '8078': 1,
 '8083': 1,
 '8090': 1,
 '8128': 1,
 '8153': 1,
 '8208': 1}

In [65]:
#switch the order order of the dictionary - now taxID: species name
switchTaxID = {y:x for x,y in taxID.iteritems()} 

glance(switchTaxID)

{'1510206': u'Salinivibrio socompensis',
 '7994': u'Astyanax mexicanus',
 '8078': u'Fundulus heteroclitus'}

In [18]:
# remember: fishDict is dictionary {taxID: (1 for fish or 0 for not fish)}
# so have two dictionaries with key=tax ID 
# use these to combine the dictionaries to get 1 {species:(1or0)}

keylist2 = fishDict.keys()
#keylist 2 is now a NONREPETITIVE list of taxIDs
fishDict2={}

for i in keylist2:
    key = switchTaxID[i]
    value = fishDict[i]
    fishDict2[key]=value

glance(fishDict2)

{u'Neolamprologus brichardi': 1,
 u'Salinivibrio socompensis': 0,
 u'Salmo salar': 1}

In [66]:
#FINALLY get dictionary with Trinity ID and 0 or 1 for FishNotFish
#GeneSpeciesDict keys are trinity ID's (only 41...)
keylist3=GeneSpeciesDict.keys()
lastDict={}
for i in keylist3:
    newval= fishDict2[(GeneSpeciesDict[i])]
    lastDict[i]= newval

len(lastDict)

41

## I screwed this up somehow in commenting and moving cells around to be in order - now only makes csv of 41 genes, not all of them.....

GeneSpeciesDict
{u'TRINITY_DN106845_c0_g1_i1': u'Larimichthys crocea',
 u'TRINITY_DN115540_c0_g1_i1': u'Vibrio campbellii',
    
taxID
 u'Fundulus heteroclitus': '8078',
 u'Gadus morhua': '8049',
 u'Haplochromis burtoni': '8153',
 u'Larimichthys crocea': '215358',
 
fishDict
{'113540': 1,
'1248232': 0,
'144197': 1,

switchTaxID
{'1510206': u'Salinivibrio socompensis',
 '7994': u'Astyanax mexicanus',
 '8078': u'Fundulus heteroclitus'}

In [34]:
len(fishDict)

20

In [67]:
with open('FishNotFish1.csv', 'w') as f:
    [f.write('{0},{1}\n'.format(key, value)) for key, value in lastDict.items()]

In [31]:
# make a dictionary for Bacteria or Eukaryote by tax id
# 0 = Bacteria, 1 = Eukaryote
fishDict = {} 
bactDict={}
keylist=lineageDict.keys()

for i in keylist:
    key = i
    value = lineageDict[key]
    if 'Bacteria' in value:
        binary = 0
    else:
        binary = 1
    bactDict[key]= binary
  
bactDict


{'113540': 1,
 '1248232': 0,
 '144197': 1,
 '1510206': 0,
 '215358': 1,
 '31033': 1,
 '32507': 1,
 '48698': 1,
 '553611': 0,
 '5854': 1,
 '680': 0,
 '7994': 1,
 '8030': 1,
 '8049': 1,
 '8078': 1,
 '8083': 1,
 '8090': 1,
 '8128': 1,
 '8153': 1,
 '8208': 1}